## Upload

In [1]:
from sqlalchemy import create_engine, text, MetaData, Table, insert
from sqlalchemy import MetaData
import pandas as pd

# engine = create_engine("mysql+pymysql://Server=db;Database=diabetes;Uid=root;Pwd=example;")
engine = create_engine("mysql+pymysql://root:example@db/diabetes?charset=utf8mb4")
metadata_obj = MetaData()

In [2]:
df = pd.read_csv("./diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
test_table = Table("bloodtest_results", metadata_obj, autoload_with=engine)
patient_table = Table("patient", metadata_obj, autoload_with=engine)
diagnosis_table = Table("diagnosis", metadata_obj, autoload_with=engine)

In [4]:
POSITIVE_ID = 1
NEGATIVE_ID = 2

In [5]:
with engine.connect() as conn:
    for i, row in df.iterrows():
        if row["Outcome"] == 1:
            did = POSITIVE_ID
        else:
            did = NEGATIVE_ID
        stmt = (
            insert(test_table).values(
                Pregnancies=row["Pregnancies"], Glucose=row["Glucose"], BloodPressure=row["BloodPressure"], SkinThickness=row["SkinThickness"], 
                Insulin=row["Insulin"], BMI=row["BMI"], DiabetesPedigreeFunction=row["DiabetesPedigreeFunction"], diagnosis_id = did
            )
        )
        result = conn.execute(stmt)
        pid = result.inserted_primary_key[0]

        stmt = (
            insert(patient_table).values(
                first_name = "",
                last_name = "",
                age = row["Age"],
                test_result_id = pid
            )
        )
        conn.execute(stmt)
    conn.commit()
    

## Download

In [1]:
from sqlalchemy import create_engine, MetaData, Table, select
from sqlalchemy import MetaData
import pandas as pd

engine = create_engine("mysql+pymysql://root:example@db/diabetes?charset=utf8mb4")
metadata_obj = MetaData()

ModuleNotFoundError: No module named 'pandas'

In [6]:
test_table = Table("bloodtest_results", metadata_obj, autoload_with=engine)
patient_table = Table("patient", metadata_obj, autoload_with=engine)
diagnosis_table = Table("diagnosis", metadata_obj, autoload_with=engine)

In [54]:
framer = []
with engine.connect() as conn:
    stmt = (
        select(test_table.c[1:], patient_table.c.age, diagnosis_table.c[1:])
        .select_from(test_table)
        .join(patient_table, test_table.c.id == patient_table.c.test_result_id)
        .join(diagnosis_table, test_table.c.diagnosis_id == diagnosis_table.c.id)
    )
    result = conn.execute(stmt)
    for row in result:
        framer.append(row._asdict())

In [56]:
df = pd.DataFrame(framer)

In [60]:
df.Outcome = df.Outcome.apply(lambda x: {"Positive": 1, "Negative": 0}[x])

In [62]:
df.to_csv("diabetes.bak.csv", index=False)